# Sumarização de texto

**Objetivo:**  
Desenvolver e comparar métodos de sumarização automática de textos, aplicando abordagens baseadas em frequência de palavras e no algoritmo TextRank, bem como avaliar a qualidade dos resumos gerados por meio de métricas automáticas ROUGE, analisando sua fidelidade e representatividade em relação ao texto original e a resumos humanos.

**Bibliotecas:**
- bs4
- collections
- heapq
- NLTK
- string
- urllib

In [ ]:
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.probability import FreqDist
from collections import defaultdict
from heapq import nlargest
from urllib.request import Request, urlopen

In [ ]:
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# 1. Sumarização baseada em Frequência de Palavras

Objetivo: Criar um resumo de um texto usando a frequência das palavras.

Passos:

· Carregar um texto grande.

· Tokenizar o texto em frases e palavras.

· Calcular a frequência das palavras ignorando stopwords.

· Criar uma pontuação para cada frase com base nas palavras mais frequentes.

· Selecionar as frases mais importantes para formar o resumo.

Dica: Utilize NLTK para tokenização e stopwords.

In [ ]:
# Carregando texto grande
link = Request("https://www.bibliaonline.com.br/acf/1co/7", headers={"User-Agent": "Mozilla/5.0"})

pagina = urlopen(link).read().decode('utf-8', 'ignore')

soup = BeautifulSoup(pagina, "lxml")
texto = soup.find(class_="FragmentView_text__g6Uq2 FragmentView_verseByVerse__l1TB0").text
print(texto)

1 Ora, quanto às coisas que me escrevestes, bom seria que o homem não tocasse em mulher; 2 Mas, por causa da fornicação, cada um tenha a sua própria mulher, e cada uma tenha o seu próprio marido. 3 O marido pague à mulher a devida benevolência, e da mesma sorte a mulher ao marido. 4 A mulher não tem poder sobre o seu próprio corpo, mas tem-no o marido; e também da mesma maneira o marido não tem poder sobre o seu próprio corpo, mas tem-no a mulher. 5 Não vos priveis um ao outro, senão por consentimento mútuo por algum tempo, para vos aplicardes ao jejum e à oração; e depois ajuntai-vos outra vez, para que Satanás não vos tente pela vossa intemperança. 6 Digo, porém, isto como que por permissão e não por mandamento. 7 Porque quereria que todos os homens fossem como eu mesmo; mas cada um tem de Deus o seu próprio dom, um de uma maneira e outro de outra. 8 Digo, porém, aos solteiros e às viúvas, que lhes é bom se ficarem como eu. 9 Mas, se não podem conter-se, casem-se. Porque é melhor cas

In [ ]:
# Tokenizar texto em frases e palavras
sentencas = sent_tokenize(texto.lower())
palavras = word_tokenize(texto.lower())

In [ ]:
# Tirando stopwords
stopwords = set(stopwords.words('portuguese') + list(punctuation))
palavras_sem_stopwords = [palavra for palavra in palavras if palavra not in stopwords]

In [ ]:
# Calcular frequencia de palavras ignorando stopwords
from nltk.probability import FreqDist
frequencia = FreqDist(palavras_sem_stopwords)
frequencia

FreqDist({'mulher': 20, 'marido': 16, 'senhor': 12, 'cada': 7, 'porém': 7, 'porque': 7, 'chamado': 7, 'deus': 6, 'coisas': 5, 'bom': 5, ...})

In [ ]:
# Criar uma pontuação para cada frase com base nas palavras mais frequentes
from collections import defaultdict
sentencas_importantes = defaultdict(int)

for i, sentenca in enumerate(sentencas):
    for palavra in word_tokenize(sentenca.lower()):
        if palavra in frequencia:
            sentencas_importantes[i] += frequencia[palavra]

sentencas_importantes

defaultdict(int,
            {0: 99,
             1: 82,
             2: 109,
             3: 24,
             4: 16,
             5: 39,
             6: 21,
             7: 5,
             8: 14,
             9: 55,
             10: 73,
             11: 55,
             12: 51,
             13: 103,
             14: 31,
             15: 52,
             16: 44,
             17: 39,
             18: 3,
             19: 14,
             20: 3,
             21: 14,
             22: 1,
             23: 17,
             24: 19,
             25: 15,
             26: 11,
             27: 70,
             28: 13,
             29: 28,
             30: 45,
             31: 22,
             32: 24,
             33: 3,
             34: 26,
             35: 22,
             36: 13,
             37: 9,
             38: 53,
             39: 8,
             40: 75,
             41: 29,
             42: 71,
             43: 27,
             44: 24,
             45: 33,
             46: 23,
           

In [ ]:
# Selecionar as frases mais importantes para formar o resumo
from heapq import nlargest
idx_sentencas_importantes = nlargest(4, sentencas_importantes, sentencas_importantes.get)

for i in sorted(idx_sentencas_importantes):
    print(sentencas[i])

1 ora, quanto às coisas que me escrevestes, bom seria que o homem não tocasse em mulher; 2 mas, por causa da fornicação, cada um tenha a sua própria mulher, e cada uma tenha o seu próprio marido.
4 a mulher não tem poder sobre o seu próprio corpo, mas tem-no o marido; e também da mesma maneira o marido não tem poder sobre o seu próprio corpo, mas tem-no a mulher.
14 porque o marido descrente é santificado pela mulher; e a mulher descrente é santificada pelo marido; de outra sorte os vossos filhos seriam imundos; mas agora são santos.
39 a mulher casada está ligada pela lei todo o tempo que o seu marido vive; mas, se falecer o seu marido fica livre para casar com quem quiser, contanto que seja no senhor.


# 2. Sumarização usando Algoritmo TextRank

Objetivo: Implementar uma sumarização automática utilizando o algoritmo TextRank.

Passos:

· Pesquisar e utilizar a biblioteca sumy para gerar um resumo.

· Aplicar o algoritmo TextRank.

· Comparar o resumo gerado com o texto original.

Dica: Utilize sumy.parsers.plaintext.PlaintextParser e sumy.summarizers.text_rank.TextRankSummarizer.

In [ ]:
!pip install sumy

In [ ]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer

In [ ]:
print("Texto original:")
print(texto)
print("")

# Aplicar textrank
parser = PlaintextParser.from_string(texto, Tokenizer("portuguese"))
summarizer = TextRankSummarizer()
resumo = summarizer(parser.document, 4)

print("Resumo gerado:")
for sentenca in resumo:
    print(sentenca)

Texto original:
1 Ora, quanto às coisas que me escrevestes, bom seria que o homem não tocasse em mulher; 2 Mas, por causa da fornicação, cada um tenha a sua própria mulher, e cada uma tenha o seu próprio marido. 3 O marido pague à mulher a devida benevolência, e da mesma sorte a mulher ao marido. 4 A mulher não tem poder sobre o seu próprio corpo, mas tem-no o marido; e também da mesma maneira o marido não tem poder sobre o seu próprio corpo, mas tem-no a mulher. 5 Não vos priveis um ao outro, senão por consentimento mútuo por algum tempo, para vos aplicardes ao jejum e à oração; e depois ajuntai-vos outra vez, para que Satanás não vos tente pela vossa intemperança. 6 Digo, porém, isto como que por permissão e não por mandamento. 7 Porque quereria que todos os homens fossem como eu mesmo; mas cada um tem de Deus o seu próprio dom, um de uma maneira e outro de outra. 8 Digo, porém, aos solteiros e às viúvas, que lhes é bom se ficarem como eu. 9 Mas, se não podem conter-se, casem-se. Por

# 3. Avaliação Automática de Resumos com ROUGE

Objetivo: Utilizar a métrica ROUGE para avaliar resumos gerados automaticamente.

Passos:

· Gerar um resumo usando qualquer método.

· Comparar com um resumo humano.

· Utilizar rouge-score para calcular ROUGE-1, ROUGE-2 e ROUGE-L.

Dica: Instale a biblioteca rouge-score (pip install rouge-score) e use rouge_scorer para calcular as métricas.

In [ ]:
!pip install rouge-score

In [ ]:
from rouge_score import rouge_scorer

In [ ]:
# Recuperando o resumo do passo anterior e concatenando as sentencas em um texto só
resumo_concatenado = ' '.join(str(sentenca) for sentenca in resumo)
print(resumo_concatenado)

4 A mulher não tem poder sobre o seu próprio corpo, mas tem-no o marido; e também da mesma maneira o marido não tem poder sobre o seu próprio corpo, mas tem-no a mulher. 11 Se, porém, se apartar, que fique sem casar, ou que se reconcilie com o marido; e que o marido não deixe a mulher. 29 Isto, porém, vos digo, irmãos, que o tempo se abrevia; o que resta é que também os que têm mulheres sejam como se não as tivessem; 30 E os que choram, como se não chorassem; e os que folgam, como se não folgassem; e os que compram, como se não possuíssem; 31 E os que usam deste mundo, como se dele não abusassem, porque a aparência deste mundo passa. 36 Mas, se alguém julga que trata indignamente a sua virgem, se tiver passado a flor da idade, e se for necessário, que faça o tal o que quiser; não peca; casem-se.


In [ ]:
# Resumo "humano" gerado para comparação
resumo_humano = """
Neste capítulo, o apóstolo Paulo dá orientações práticas sobre casamento, celibato e relacionamentos à luz da fé cristã. Ele afirma que o casamento é uma forma legítima de evitar a imoralidade sexual e ensina que marido e mulher devem cumprir seus deveres conjugais com amor e respeito mútuo.
Paulo recomenda que, se possível, as pessoas permaneçam solteiras para se dedicarem mais plenamente a Deus, mas reconhece que o casamento é uma escolha correta e necessária para muitos. Ele também aborda situações em que um cônjuge é descrente, incentivando a manutenção do casamento se houver convivência pacífica.
Ao longo do capítulo, Paulo reforça que cada um deve viver de acordo com a condição em que foi chamado por Deus, seja solteiro, casado ou viúvo. Seu conselho é guiado por um desejo de poupar os cristãos de dificuldades desnecessárias e ajudá-los a viver com foco no Senhor, especialmente diante da brevidade da vida.
"""

In [ ]:
# Utilizar rouge-score para calcular ROUGE-1, ROUGE-2 e ROUGE-L
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(resumo_humano, resumo_concatenado)

print("Métricas ROUGE:")
for tipo, resultado in scores.items():
    print(f"{tipo.upper()}:")
    print(f"  Precisão: {resultado.precision:.4f}")
    print(f"  Revocação: {resultado.recall:.4f}")
    print(f"  F1-Score: {resultado.fmeasure:.4f}")

Métricas ROUGE:
ROUGE1:
  Precisão: 0.2663
  Revocação: 0.2917
  F1-Score: 0.2784
ROUGE2:
  Precisão: 0.0273
  Revocação: 0.0299
  F1-Score: 0.0286
ROUGEL:
  Precisão: 0.1467
  Revocação: 0.1607
  F1-Score: 0.1534
